In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import shutil

import matplotlib.pyplot as plt
from PIL import Image
from skimage import io
from skimage.transform import resize
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf

## Read dataset

In [3]:
# Main Directory
main_dir = 'Dataset/5001'
print(f'The total number of images is {len(os.listdir(main_dir))}')

The total number of images is 165


In [4]:
# specify the img directory path
path = main_dir

# list files in img directory
files = os.listdir(path)

image_list = []

for file in files:
    # make sure file is an image
    if file.endswith(('.jpg', '.png', 'jpeg', 'tiff')):
        #img_path = path + file
        image_list.append(os.path.join(path, file))
        
print(f'Image list top 5 examples:')
image_list.sort(reverse=False)
image_list[:5]

Image list top 5 examples:


['Dataset/5001/image_2015-11-01.tiff',
 'Dataset/5001/image_2015-11-08.tiff',
 'Dataset/5001/image_2015-11-15.tiff',
 'Dataset/5001/image_2015-11-22.tiff',
 'Dataset/5001/image_2015-11-29.tiff']

In [5]:
target_size = (740, 740, 12)

# VAE

In [6]:
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, MaxPooling2D, UpSampling2D
#from keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

In [7]:
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

## Encoder Part

In [8]:
input_data = Input(shape=target_size, name='encoder_input')

encoder = Conv2D(32, (5,5), activation='relu')(input_data)
encoder = MaxPooling2D((2,2))(encoder)

encoder = Conv2D(64, (3,3), activation='relu')(encoder)
encoder = MaxPooling2D((2,2))(encoder)

encoder = Conv2D(64, (3,3), activation='relu')(encoder)
encoder = MaxPooling2D((2,2))(encoder)

conv_shape = K.int_shape(encoder) #Shape of conv to be provided to decoder

encoder = Flatten()(encoder)
encoder = Dense(200, activation='relu')(encoder)

## Latent Distribution and Sampling

In [9]:
def sample_latent_features(distribution):
    distribution_mean, distribution_variance = distribution
    batch_size = tf.shape(distribution_variance)[0]
    random = K.random_normal(shape=(batch_size, tf.shape(distribution_variance)[1]))
    return distribution_mean + tf.exp(0.5 * distribution_variance) * random

In [10]:
latent_dim = 100 # Number of latent dim parameters

distribution_mean = Dense(latent_dim, name='mean')(encoder)
distribution_variance = Dense(latent_dim, name='log_variance')(encoder)
latent_encoding = Lambda(sample_latent_features)([distribution_mean, distribution_variance])

In [11]:
encoder_model = Model(input_data, latent_encoding)
encoder_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 740, 740, 1  0           []                               
                                2)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 736, 736, 32  9632        ['encoder_input[0][0]']          
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 368, 368, 32  0           ['conv2d[0][0]']                 
                                )                                                             

## Decoder Part

In [12]:
decoder_input = Input(shape=(latent_dim), name='decoder_input')
decoder = Dense(conv_shape[1]*conv_shape[2]*conv_shape[3])(decoder_input)

decoder = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(decoder)

"""
encoder = Conv2D(64, (5,5), activation='relu')(input_data)
encoder = MaxPooling2D((2,2))(encoder)

encoder = Conv2D(64, (3,3), activation='relu')(encoder)
encoder = MaxPooling2D((2,2))(encoder)

encoder = Conv2D(32, (3,3), activation='relu')(encoder)
encoder = MaxPooling2D((2,2))(encoder)
"""

decoder = UpSampling2D((2,2))(decoder)
decoder = Conv2DTranspose(64, (3,3), activation='relu')(decoder)

decoder = UpSampling2D((2,2))(decoder)
decoder = Conv2DTranspose(64, (3,3), activation='relu')(decoder)

decoder = Conv2DTranspose(32, (3,3), activation='relu')(decoder)
decoder = UpSampling2D((2,2))(decoder)

decoder_output = Conv2DTranspose(target_size[2], (5,5), activation='relu')(decoder)

In [13]:
decoder_model = Model(decoder_input, decoder_output)
decoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 100)]             0         
                                                                 
 dense_1 (Dense)             (None, 518400)            52358400  
                                                                 
 reshape (Reshape)           (None, 90, 90, 64)        0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 180, 180, 64)     0         
 )                                                               
                                                                 
 conv2d_transpose (Conv2DTra  (None, 182, 182, 64)     36928     
 nspose)                                                         
                                                                 
 up_sampling2d_1 (UpSampling  (None, 364, 364, 64)     0   

## Combining

In [14]:
encoded = encoder_model(input_data)
decoded = decoder_model(encoded)

In [15]:
autoencoder = Model(input_data, decoded)

## Loss Function (Reconstruction Loss + KL-loss)
https://towardsdatascience.com/variational-autoencoders-as-generative-models-with-keras-e0c79415a7eb

In [16]:
def get_loss(distribution_mean, distribution_variance):
    
    def get_reconstruction_loss(y_true, y_pred):
        reconstruction_loss = keras.losses.mse(y_true, y_pred)
        reconstruction_loss_batch = tf.reduce_mean(reconstruction_loss)
        return reconstruction_loss_batch#*target_size[0]*target_size[1]
    
    def get_kl_loss(distribution_mean, distribution_variance):
        kl_loss = 1 + distribution_variance - tf.square(distribution_mean) - tf.exp(distribution_variance)
        kl_loss_batch = tf.reduce_mean(kl_loss)
        #return kl_loss_batch*(-5e-4)
        return kl_loss_batch*(-.5)
    
    def total_loss(y_true, y_pred):
        reconstruction_loss_batch = get_reconstruction_loss(y_true, y_pred)
        kl_loss_batch = get_kl_loss(distribution_mean, distribution_variance)
        #return reconstruction_loss_batch + kl_loss_batch
        return K.mean(reconstruction_loss_batch + kl_loss_batch)
    
    return total_loss

## Compile the model

In [17]:
# Compile VAE
autoencoder.compile(loss=get_loss(distribution_mean, distribution_variance), optimizer='adam')
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 740, 740, 12)]    0         
                                                                 
 model (Functional)          (None, 100)               103785456 
                                                                 
 model_1 (Functional)        (None, 740, 740, 12)      52460332  
                                                                 
Total params: 156,245,788
Trainable params: 156,245,788
Non-trainable params: 0
_________________________________________________________________


# Model of 100 features

In [18]:
autoencoder.load_weights('Models/vae.h5', by_name=True)

Metal device set to: Apple M1


2022-06-05 10:51:23.284272: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-05 10:51:23.284581: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-06-05 10:51:23.300908: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-05 10:51:23.302016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-05 10:51:23.340814: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-05 10:51:23.662002: I tensorflow/core/grappler/o

# Generate embeddings

In [19]:
model = keras.Sequential()
for layer in autoencoder.layers[:-1]: # just exclude last layer from copying
    model.add(layer)
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 100)               103785456 
                                                                 
Total params: 103,785,456
Trainable params: 103,785,456
Non-trainable params: 0
_________________________________________________________________


In [20]:
def read_image(path):
    image_test = io.imread(path)
    image_test = resize(image_test, (target_size[0], target_size[1]),
                           anti_aliasing=True)
    image_test = np.expand_dims(image_test,axis=0)
    return image_test

In [21]:
def generate_embedding(image):
    embaedding = model.predict(image)
    embaedding = np.squeeze(embaedding, axis=0)
    return embaedding

In [22]:
def get_image_name(path):
    image_name = path[path.index('/image')+7:path.index('.tiff')]
    return image_name

In [23]:
embeddings = pd.DataFrame(columns=['Date', 'Embedding'])

for path in image_list:
    image = read_image(path=path)
    embedding = generate_embedding(image)
    date = get_image_name(path=path)
    embeddings = embeddings.append({'Date': date, 'Embedding': embedding}, ignore_index=True )

embeddings

2022-06-05 10:51:24.518691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Date,Embedding
0,2015-11-01,"[-0.77316415, 0.40942943, -1.9674243, -0.19426..."
1,2015-11-08,"[-0.37579188, 1.2821914, -1.7693113, -1.768404..."
2,2015-11-15,"[0.27006036, 1.8966693, 0.16080439, 0.10046702..."
3,2015-11-22,"[-1.0504739, -0.4924668, -3.0571184, -2.102062..."
4,2015-11-29,"[0.27709633, -0.17730333, 1.5202425, 1.1475053..."
...,...,...
160,2018-11-25,"[-0.2209534, 1.9359521, -2.9964356, -0.2979858..."
161,2018-12-02,"[1.3303635, -0.22096978, -2.2959766, -5.403095..."
162,2018-12-09,"[-0.37533164, -1.1416743, -0.72453666, -0.6877..."
163,2018-12-16,"[-0.8458848, -3.2026246, -1.2519635, -0.646646..."


In [24]:
def split_columns(df):
    df_aux = pd.DataFrame(df['Embedding'].tolist())
    df_aux = pd.concat( [df['Date'], df_aux], axis=1)
    return df_aux

In [25]:
# new df from the column of lists
embeddings_df = split_columns(embeddings)

embeddings_df.to_csv('Embeddings/embeddings_medellin_100features.csv',index=False)
# display the resulting df
embeddings_df

,Date,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,2015-11-01,-0.773164,0.409429,-1.967424,-0.194265,1.589418,-0.806473,3.013131,-0.131107,-1.274766,...,1.858297,-1.143137,-0.012872,2.545450,-1.408913,2.835881,-4.650385,2.829885,-3.892387,-2.027068
1,2015-11-08,-0.375792,1.282191,-1.769311,-1.768404,0.960811,-1.401841,-0.138934,0.260522,-1.184208,...,1.427782,-2.927658,0.504012,-0.062580,3.175248,-1.147624,-2.653756,2.155063,-1.433321,-1.319588
2,2015-11-15,0.270060,1.896669,0.160804,0.100467,-1.593640,-0.673158,0.158351,1.709716,-0.976310,...,1.151502,-1.066319,0.433531,0.623669,1.678527,-2.044615,0.735826,1.307580,-0.059874,-0.276768
3,2015-11-22,-1.050474,-0.492467,-3.057118,-2.102062,0.290074,-1.837421,0.771637,0.883183,0.827531,...,1.586739,-0.414671,-0.480104,-1.629912,3.934243,2.386098,-1.041708,0.388762,0.855838,-1.768747
4,2015-11-29,0.277096,-0.177303,1.520242,1.147505,0.999619,0.343843,1.057434,1.925118,0.395960,...,1.060055,-0.819456,-0.657092,0.556108,3.519002,-1.769988,-2.901545,0.582610,3.461969,-1.540928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2018-11-25,-0.220953,1.935952,-2.996436,-0.297986,0.559725,-1.316638,0.127099,0.808687,-0.865521,...,1.097104,-1.500260,0.139145,0.247580,1.519686,0.246774,-1.867187,1.393556,3.392008,-0.358894
161,2018-12-02,1.330364,-0.220970,-2.295977,-5.403095,-0.123737,-1.548519,1.949868,0.354802,1.288355,...,1.961233,-2.454971,-0.320453,0.415422,4.667178,1.379886,-1.097326,2.208820,1.959872,-2.071136
162,2018-12-09,-0.375332,-1.141674,-0.724537,-0.687713,-1.182274,-0.982378,1.187518,-1.785588,-0.709939,...,1.029219,-2.047943,-0.039943,1.954097,7.794002,1.010624,-1.740980,0.549338,-0.777285,-1.311492
163,2018-12-16,-0.845885,-3.202625,-1.251963,-0.646646,-0.755027,-0.733298,-0.165185,-0.394726,-2.383777,...,0.604045,-3.127710,-0.387775,2.545238,-4.183140,0.834891,-2.535359,2.277708,-0.649933,-2.139267


In [26]:
pd.read_csv('Embeddings/embeddings_medellin_100features.csv')

,Date,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,2015-11-01,-0.773164,0.409429,-1.967424,-0.194265,1.589418,-0.806473,3.013131,-0.131107,-1.274766,...,1.858297,-1.143137,-0.012872,2.545450,-1.408913,2.835881,-4.650385,2.829885,-3.892387,-2.027068
1,2015-11-08,-0.375792,1.282191,-1.769311,-1.768404,0.960811,-1.401841,-0.138934,0.260522,-1.184208,...,1.427782,-2.927658,0.504012,-0.062580,3.175248,-1.147623,-2.653756,2.155063,-1.433320,-1.319588
2,2015-11-15,0.270060,1.896669,0.160804,0.100467,-1.593640,-0.673158,0.158351,1.709716,-0.976310,...,1.151502,-1.066319,0.433531,0.623669,1.678527,-2.044615,0.735826,1.307580,-0.059874,-0.276768
3,2015-11-22,-1.050474,-0.492467,-3.057118,-2.102062,0.290074,-1.837421,0.771637,0.883183,0.827531,...,1.586739,-0.414671,-0.480104,-1.629912,3.934243,2.386098,-1.041708,0.388762,0.855838,-1.768747
4,2015-11-29,0.277096,-0.177303,1.520242,1.147505,0.999619,0.343843,1.057434,1.925118,0.395960,...,1.060054,-0.819456,-0.657092,0.556108,3.519002,-1.769988,-2.901545,0.582610,3.461969,-1.540928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2018-11-25,-0.220953,1.935952,-2.996436,-0.297986,0.559725,-1.316637,0.127099,0.808687,-0.865521,...,1.097104,-1.500260,0.139145,0.247580,1.519686,0.246774,-1.867187,1.393556,3.392008,-0.358893
161,2018-12-02,1.330364,-0.220970,-2.295977,-5.403095,-0.123737,-1.548519,1.949868,0.354802,1.288355,...,1.961233,-2.454971,-0.320453,0.415422,4.667178,1.379886,-1.097326,2.208820,1.959872,-2.071136
162,2018-12-09,-0.375332,-1.141674,-0.724537,-0.687713,-1.182274,-0.982379,1.187518,-1.785588,-0.709939,...,1.029219,-2.047943,-0.039943,1.954097,7.794002,1.010624,-1.740980,0.549338,-0.777285,-1.311492
163,2018-12-16,-0.845885,-3.202625,-1.251964,-0.646647,-0.755027,-0.733298,-0.165185,-0.394726,-2.383777,...,0.604045,-3.127710,-0.387775,2.545238,-4.183140,0.834891,-2.535359,2.277708,-0.649933,-2.139267
